### Assignment : Week 3
## Efficiently finding optimal policies in MABs

In this assignment, we will work with Multi Armed Bandit environments, and try to find the best policies using different strategies to minimize the total regret.

The aim of this exercise is to code agents capable of understanding the underlying probability distributions of the environment and finding the most optimal actions as early as possible.

You can start this assignment during/after reading Grokking Ch-4.

Let's get started!

In [39]:
# importing necessary stuff
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

# if you want to use envs from Gym, import it
# import gym, gym_bandits

Let's make a simple **2-armed Bernoulli** bandit.

If you want a cleaner code, you can implement Bandits using `class` in Python.

We have included sample code for this in `bandits.py` which you can take/import.

In [40]:
class Bernoulli_bandit():
    def __init__(self, n):
        self.N = n
        self.mean = np.random.random()
        self.probs = np.random.normal(self.mean, 1, (n,))
        self.probs = np.random.rand(n)
        self.optimal = np.max(self.probs)
        self.regret = 0.0
        
    def evaluate_choice(self, k):
        assert 1 <= k <= self.N
        reward = np.random.binomial(1, self.probs[k-1])
        self.regret += self.optimal - reward
        return reward

In [41]:
class Gaussian_bandit():
    def __init__(self, n):
        self.N = n
        self.means = np.random.rand(n)
        self.variances = np.sqrt(means)*(np.random.rand(n)/2 + 0.5)
        self.optimal = np.max(self.means)
        self.regret = 0.0

    def evaluate_choice(self, k):
        assert 1 <= k <= self.N
        reward = np.random.uniform(self.means[k-1], self.variances[k-1]) 
        self.regret += np.random.uniform(self.optimal, self.variances[k-1]) - reward
        return reward

Let's now make a template for testing different strategies.

Feel free to modify this code.

In [42]:
# strategy function takes in the environment function, number of actions, and a selector function
# it also takes in the number of episodes to run the strategy for (higher episodes = more accurate Q values)

def test_strategy(env, n_actions, selector, n_episodes = 1000):
    
    # initialize Q and N to 0s
    Q = np.zeros(n_actions)
    N = np.zeros(n_actions)

    # loop for n_episodes
    for e in tqdm(range(n_episodes)):
        
        # selector function takes in current Q and returns an action
        # modify the selector function according to the strategy
        action = selector(Q)

        # get the reward from the environment
        reward = env(action)

        # update N and Q
        N[action] += 1
        Q[action] += (reward - Q[action])/N[action]

    # return the best action
    return np.argmax(Q)

Implementing the simplest selector using pure-exploration strategy.

In [43]:
# selector returns a random action
random_selector = lambda Q : np.random.randint(len(Q))

Let's test this strategy.

In [ ]:
test_strategy(mab_2_env, 2, random_selector)

As you can see, it returns the optimal action. Let's check if that's indeed true.

We can do that by revealing the actual `probs` distribution.

In [45]:
probs

array([0.80499935, 0.65065254])

As expected, our pure exploration strategy does indeed return the optimal action for this Bernoulli bandit. 

You can try generating new bandits with different `probs` and try out the same.

With all this in place, here's what you have to do -

Recall that, the regret $\mathcal{T}$ is given by,

$$\mathcal{T}=\sum  _{e=1} ^{E} \mathbb{E} \left[ v_* - q_* \left( A_e \right) \right]$$

We can only calculate it when we have the $v_*$ and $q_*$ functions known beforehand. Since we are making the MDPs from scratch, that's not an issue for us right now.

But remember, in real-life problems, these functions are not known. Hence we must be aware of multiple policy finding strategies and try the one which gives best results fastest.

### Todo 0

Implement the calculation of the total regret $\mathcal{T}$ for your strategy.

To do this, you will need to store the rewards obtained each episode. Modify the `strategy` function accordingly.

In [46]:
# function to calculate total regret

def regret(rewards, probs):

    # implement the regret function here
    return len(rewards)*np.max(probs) - sum(rewards)

### Todo 1

Now, let's implement some other selection strategies and compare their regret with the simple exploration strategy.

Note that some of these strategies involve hyperparameter(s) which need to be manually adjusted. You have to play around with the values and see which one gives you best results.

This is known as "hyperparameter tuning" and is quite commonly done while working with complex models (including neural networks). Personally, you should try out some natural values (including the ones given in the book) along with some extreme values where it is easy to manually verify the correctness of your strategy.

In [47]:


def epsilon_greedy(Q, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

In [48]:
# exponentially decaying epsilon greedy strategy

def exponentially_decaying_epsilon_greedy(Q, epsilon, episode):
    
    # implement the exponentially decaying epsilon greedy strategy here
    epsilon *= episode
    if np.random.random() < epsilon:
        return np.random.randint(1, len(Q) + 1), epsilon
    return np.argmax(Q) + 1, epsilon

In [49]:
# softmax action selection strategy

def softmax_strategy(Q, temperature):
    
    # implement the softmax strategy here
    choice_probabilities = [np.exp(val/temperature) for val in Q]
    choice_probabilities = choice_probabilities/sum(choice_probabilities)

    return np.random.choice(range(1, len(Q)+1), p=choice_probabilities)


In [50]:
# upper confidence bound strategy
from math import log, log10

def ucb(Q, N, c):
    
    # implement the ucb strategy here
    s = sum(N)
    choice = np.argmax([Q[i] + np.sqrt(log(s)/N[i]) if N[i] >= 1 else Q[i] for i in range(len(Q))]) + 1
    return choice

In [51]:
# thompson sampling strategy

def thompson_sampling(Q, N, a, b):
    samples = np.random.normal(loc=Q, scale = a/(np.sqrt(N) + b))
    choice = np.argmax(samples) + 1
    return choice

### Todo 2

Run each strategy for 2-armed bandit environment and compare the total regrets.

You can also try plotting the regret vs episode graph and check if it matches the expected result from Grokking

In [52]:
# import plotting libraries

import matplotlib.pyplot as plt

In [ ]:
bandit = Bernoulli_bandit(10)
bandit.probs

In [ ]:
def implement_epsilon_greedy(epsilon=0.1, gamma=0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = epsilon_greedy(Q, epsilon)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma*bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q) + 1

In [ ]:
def implement_decaying_epsilon_greedy(epsilon, gamma=0.9, num_episodes = 10000):
    global bandit
    min_to_max_ratio = 0.01
    max_epsilon = epsilon
    min_epsilon = min_to_max_ratio*epsilon
    decay_rate = 0.05
    decay_episodes = int(num_episodes*decay_rate)
    remaining_episodes = num_episodes - decay_episodes
    epsilons = 0.01
    epsilons /= np.logspace(log10(min_to_max_ratio), 0, decay_episodes)
    epsilons *= max_epsilon - min_epsilon
    epsilons += min_epsilon
    epsilons = np.pad(epsilons, (0, remaining_episodes), mode='constant', constant_values=min_epsilon)
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = epsilon_greedy(Q, epsilons[episode])
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma*bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
        
    return np.argmax(Q) + 1

In [ ]:
def implement_softmax(temperature, gamma=0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = softmax_strategy(Q, temperature)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma*bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q) + 1

In [ ]:
def implement_ucb(c, gamma=0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.ones(bandit.N)
    for episode in range(num_episodes):
        choice = ucb(Q, N, c)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma*bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q) + 1

In [ ]:
def implement_thompson_sampling(alpha=1, beta=0, gamma = 0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = thompson_sampling(Q, N, alpha, beta)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma*bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q) + 1

In [ ]:
epsilon_values = np.linspace(0, 1, 101)
regrets = []
choices = []
for epsilon in epsilon_values:
    optimal_choice = implement_decaying_epsilon_greedy(epsilon)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.xlabel('Epsilon')
plt.title('Exponentially Decaying Epsilon Greedy Policy')
plt.scatter(epsilons, regrets)

In [ ]:
# epsilons = np.linspace(0, 1, 101)
epsilon_values = [0.1]
values = []
rewards = []
results = []
for i in range(50):
    band = Bernoulli_bandit(n)
    epsilon = epsilon_values[0]
    n = 2
    band.regret = 0
    gamma = 0.9
    gamma_power = 0.9
    Q = np.zeros(n)
    for episode in range(10000):
        choice = epsilon_greedy(Q, epsilon)
        Q[choice-1] += gamma_power*band.evaluate_choice(choice)
        gamma_power *= gamma
    rewards.append(band.regret)
    results.append(abs(band.probs[0] - band.probs[1])/band.regret)

In [ ]:
epsilons = np.linspace(0, 1, 101)
regrets = []
for epsilon in epsilons:
    bandit.regret = 0
    gamma = 0.9
    gamma_power = 0.9
    Q = np.zeros(n)
    N  = np.zeros(n)
    for episode in range(10000):
        choice = epsilon_greedy(Q, epsilon)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma*bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    bandit.regret = 0
    choice = np.argmax(Q) + 1
    for episode in range(10000):
        bandit.evaluate_choice(choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('Epsilon Greedy Policy')
plt.xlabel('Epsilon')
plt.scatter(epsilons, regrets)

In [ ]:
temperature_values = np.linspace(0, 1, 101)[1:]
regrets = []
choices = []
for temp in temperature_values:
    optimal_choice = implement_softmax(temp)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('Softmax Policy')
plt.xlabel('Epsilon')
plt.scatter(temperature_values, regrets)

In [ ]:
c_values = np.linspace(0, 1, 101)
regrets = []
choices = []
for c in c_values:
    optimal_choice = implement_ucb(c)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('UCB Policy')
plt.xlabel('c')
plt.scatter(c_values, regrets)

### Todo 3

The 2-armed bandits might be too simple for us to actually see substantial difference in the regret of these strategies. 

Let's now create a more complicated bandit environment and replicate our results on it.

We will now implement a 10-armed Gaussian bandit. 

As required, it will have possible actions and each action will generate a reward sampled from a Gaussian distribution.

Hence, each "arm" will have a randomly generated $\mu$ and $\sigma$, and the rewards will be generated with probabilities following the $\mathcal{N}(\mu, \sigma^2)$ distribution. 

In [53]:
# 10 arm gaussian bandit
n = 10
means = 10*np.random.rand(n)

variances = np.sqrt(means)*np.random.rand(n)

In [54]:
bandit = Gaussian_bandit(n)
bandit.means

### Todo 4

Test the different strategies on the 10-armed gaussian bandit and verify your results.

In [ ]:
epsilon_values = np.linspace(0, 1, 101)
regrets = []
choices = []
for epsilon in epsilon_values:
    optimal_choice = implement_epsilon_greedy(epsilon)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('Epsilon Greedy Policy')
plt.xlabel('Epsilon')
plt.scatter(epsilons, regrets)

In [ ]:
epsilon_values = np.linspace(0, 1, 101)
regrets = []
choices = []
for epsilon in epsilon_values:
    optimal_choice = implement_decaying_epsilon_greedy(epsilon)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('Decaying Epsilon Greedy Policy')
plt.xlabel('Epsilon')
plt.scatter(epsilons, regrets)

In [ ]:
temperature_values = np.linspace(0, 1, 101)[1:]
regrets = []
choices = []
for temp in temperature_values:
    optimal_choice = implement_softmax(temp)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('Softmax Policy')
plt.xlabel('Epsilon')
plt.scatter(temperature_values, regrets)

In [ ]:
c_values = np.linspace(0, 1, 101)
regrets = []
choices = []
for c in c_values:
    optimal_choice = implement_ucb(c)
    bandit.regret = 0
    print(optimal_choice)
    for episode in range(1000):
        bandit.evaluate_choice(optimal_choice)
    regrets.append(bandit.regret)

In [ ]:
plt.ylabel('Regret')
plt.title('UCB Policy')
plt.xlabel('c')
plt.scatter(c_values, regrets)